In [1]:
#librerias a importar

#para que el usuario interactue
from ipywidgets import interact, interactive, interact_manual

#Para mate y graficos
import numpy as np
import matplotlib.pyplot as plt
from skimage.util.shape import view_as_blocks
from PIL import Image


#Para procesar imagenes en python
from skimage import data
from skimage.color import rgb2gray
from  skimage import img_as_ubyte,img_as_float
from skimage.transform import resize


#para realizar el svd
from numpy.linalg import svd

In [2]:
%matplotlib inline

In [3]:
##para leer la imagen
##Imagen 1
I = plt.imread('quetzal.png')[:,:,:3]
I.shape

(900, 1200, 3)

In [4]:
##Imagen 2
II = plt.imread('flores.png')[:,:,:3]
II.shape

(675, 1200, 3)

In [5]:
##pasamos las imagenes a gris para el svd
Igray = rgb2gray(I)
Igray.shape

IIgray = rgb2gray(II)

In [6]:
##hacer mis imagenes cuadradas
def make_square(image):
    if len(image.shape) == 2:  # Imagen en escala de grises
        height, width = image.shape
    elif len(image.shape) == 3:  # Imagen en color
        height, width, _ = image.shape
    else:
        raise ValueError("Formato de imagen no válido. Se esperaba una imagen en escala de grises (2D) o en color (3D).")

    min_dim = min(height, width)

    # Calcular las coordenadas de corte para recortar la imagen
    start_row = (height - min_dim) // 2
    end_row = start_row + min_dim
    start_col = (width - min_dim) // 2
    end_col = start_col + min_dim

    # Recortar la imagen para hacerla cuadrada
    square_image = image[start_row:end_row, start_col:end_col]

    return square_image


In [7]:
##vamos a pasar todas nuestras imagenes a un diccionario para poder utilizarlas despues
gray_images = {"quetzal": (img_as_float(Igray)), "flores": (img_as_float(IIgray))}
color_images = {"quetzal": (img_as_float(I)), "flores": (img_as_float(II))}

##volvemos las imagenes cuadradas
for key, image in gray_images.items():
    gray_images[key] = make_square(image)
for key, image in color_images.items():
    color_images[key] = make_square(image)

## Descomposición SVD

In [8]:
##para hacer el svd de las imagenes
def compress_svd(image,k, b):
    U, S, V = svd(image)
    reconst_matrix = np.dot(U[:, :k], np.dot(np.diag(S[:k]), V[:k, :])) #reconstruimos la matriz
    return reconst_matrix, S, U, V


## Compresión de imagenes en gris

In [13]:
def compresiongris(Imagen, k, b):
    
    image = gray_images[Imagen]
    original_shape = image.shape
    
    # Dividimos la imagen en bloques de tamaño b
    num_blocks = image.shape[0] // b
    image_blocks = [image[i*b:(i+1)*b, j*b:(j+1)*b] for i in range(num_blocks) for j in range(num_blocks)]
    
    compressed_blocks = []
    U_blocks = []
    S_blocks = []
    V_blocks = []
    
    for block in image_blocks:
        # Aplicamos SVD a cada bloque
        reconst_block, S, U, V = compress_svd(block, k, b)
        compressed_blocks.append(reconst_block)
        U_blocks.append(U)
        S_blocks.append(S)
        V_blocks.append(V)
    
    # Reconstruimos la imagen a partir de los bloques comprimidos
    reconst_image = np.zeros_like(image)
    for i in range(num_blocks):
        for j in range(num_blocks):
            reconst_image[i*b:(i+1)*b, j*b:(j+1)*b] = compressed_blocks[i*num_blocks + j]
    
    compress_ratio = 100.0*(4* ((I.shape[0] * k) + k + (k * I.shape[1])))/(1024**2)
    plt.title('Radio de compresion = {:.2f}'.format(compress_ratio) + "%")
    plt.imshow(reconst_image, cmap='gray')

In [14]:
interact(compresiongris, Imagen=list(gray_images.keys()),k=(1,100), b = (1, 50)) ##creamos una interfaz para poder cambiar a nuestro gusto la k y b. 

interactive(children=(Dropdown(description='Imagen', options=('quetzal', 'flores'), value='quetzal'), IntSlide…

<function __main__.compresiongris(Imagen, k, b)>

## Compresión de imagenes a color

In [53]:
def compresioncolor(Imagen, k, b):
    image = color_images[Imagen] ##llamamos a la imagen con color\n",
    original_shape = image.shape
    image_reshaped = image.reshape((original_shape[0],original_shape[1]*3))
    image_reconst,S, V, U =compress_svd(image_reshaped,k, b) # Realizamos la compresión SVD en la imagen con la cantidad de K
    image_reconst = image_reconst.reshape(original_shape) # Volvemos a dar forma a la imagen reconstruida a la forma original
    # Calculamos la relación de compresión en porcentaje utilizando la fórmula especificada. (aun no se si si es esta vdd)
    compress_ratio = 100.0*(k*(original_shape[0]+ 3*original_shape[1]+k))/(original_shape[0]*original_shape[1]*original_shape[2]) 
    plt.title('radio de compresion = {:.2f}'.format(compress_ratio) + "%")
    plt.imshow(image_reconst)


In [55]:
interact(compresioncolor, Imagen=list(color_images.keys()),k=(1,50), b=(1,50)) ##creamos una interfaz para poder cambiar a nuestro gusto la k y b. 

interactive(children=(Dropdown(description='Imagen', options=('quetzal', 'flores'), value='quetzal'), IntSlide…

<function __main__.compresioncolor(Imagen, k, b)>